In [ ]:
ls -1 /content/drive/MyDrive/AI_Rayan/P2/Q1/Dataset/train/images | wc -l


4000


In [ ]:
ls -1 /content/drive/MyDrive/AI_Rayan/P2/Q1/Dataset/val/images | wc -l

115


# Install

In [ ]:
%load_ext autoreload
%autoreload 2
#%env CUDA_VISIBLE_DEVICES=3

In [ ]:
colab_requirements = [
    "pip install librosa",
    "pip install noisereduce",
    "pip install soundfile",

]

import sys, subprocess

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    for i in colab_requirements:
        run_subprocess_command(i)

In [ ]:
! pip install pydub

# Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing - Dataset

## Preprocessing

## Dataset (TheDataset)

In [ ]:
import os
from functools import cmp_to_key
import re
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image

class TheDataset(Dataset):
    def __init__(
        self,
        root_dir,
        chunk_size,
        sample_rate,
        exclude_list=None,
        transform=None,
        with_labels=True
    ):
        self.root_dir = root_dir
        self.chunk_size = chunk_size
        self.sample_rate = sample_rate
        self.transform = transform
        self.with_labels = with_labels
        self.dataset = []

        # If exclude_list is not provided, initialize it as an empty list
        if exclude_list is None:
            exclude_list = []

        # Get all subdirectories excluding those in the exclude_list
        subdirs = sorted([d for d in os.listdir(root_dir)
                          if os.path.isdir(os.path.join(root_dir, d)) and d not in exclude_list])

        # Initialize lists to store all PNG filenames
        all_pngs = []

        # Iterate through subdirectories
        for subdir in subdirs:
            cur_fol = os.path.join(self.root_dir, subdir)
            png_files = self.sort_files([f for f in os.listdir(cur_fol) if f.lower().endswith('.png')])
            png_files = [os.path.join(cur_fol, f) for f in png_files]

            rewrited = self.rewrite(png_files)
            all_pngs.extend(rewrited)

        self.dataset = all_pngs


    def downsample_labels(self, labels, num_seconds, sample_rate, threshold=0.05):
      downsampled_labels = np.zeros(num_seconds, dtype=int)  # Initialize output array

      for i in range(num_seconds):
          start_index = i * sample_rate
          end_index = start_index + sample_rate

          # Get the current second's labels
          current_second_labels = labels[start_index:end_index]

          # Check if more than 5% of the samples are 1s
          if np.mean(current_second_labels) > threshold:
              downsampled_labels[i] = 1  # Flag as 1 if over threshold
      return downsampled_labels

    def rewrite(self, array):
      output = []
      for i in range(len(array)-2):
        mini = array[i:i+3]
        output.append(mini)

      return output



    def sort_files(self, file_list):
        def extract_number(filename):
            match = re.match(r'chunk_(\d+)\.png$', filename.lower())
            return int(match.group(1)) if match else float('inf')

        def custom_key(file1, file2):
            num1 = extract_number(file1)
            num2 = extract_number(file2)
            if num1 == num2:
                return file1.lower().compare(file2.lower())
            return num1 - num2

        return sorted(file_list, key=cmp_to_key(custom_key))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        chunk = self.dataset[idx]
        img_before = chunk[0]
        img_current = chunk[1]
        img_after = chunk[2]

        label_name_current = chunk[1][:-4] + ".npy"
        label_current = np.load(label_name_current)

        mel_img_before = Image.open(img_before).convert('RGB')  # Convert to RGB for CNN
        mel_img_current = Image.open(img_current).convert('RGB')  # Convert to RGB for CNN
        mel_img_after = Image.open(img_after).convert('RGB')  # Convert to RGB for CNN


        # Apply any transformations (e.g., resizing, normalization)
        if self.transform:
            mel_img_before = self.transform(mel_img_before)
            mel_img_current = self.transform(mel_img_current)
            mel_img_after = self.transform(mel_img_after)


        if self.with_labels:
          labels = self.downsample_labels(label_current, self.chunk_size, self.sample_rate)
          labels = torch.tensor(labels, dtype=torch.float32)

          return (mel_img_before,mel_img_current,mel_img_after), labels

        return (mel_img_before,mel_img_current,mel_img_after)



# Model (TheModel)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define a Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = F.relu(out)
        return out

# Define the Mini ResNet for each image
class MiniResNet(nn.Module):
    def __init__(self, num_labels):
        super(MiniResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(64, 64, num_blocks=2, stride=1)
        self.layer2 = self._make_layer(64, 128, num_blocks=2, stride=2)
        self.layer3 = self._make_layer(128, 256, num_blocks=2, stride=2)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

# Define the final network that processes 3 images separately and concatenates the outputs
class TheModel(nn.Module):
    def __init__(self, num_labels):
        super(TheModel, self).__init__()

        # The MiniResNet used to process each individual image
        self.resnet_branch1 = MiniResNet(num_labels)
        self.resnet_branch2 = MiniResNet(num_labels)
        self.resnet_branch3 = MiniResNet(num_labels)

        # Fully connected layer after concatenating outputs from 3 branches
        self.fc = nn.Sequential(
            nn.Linear(256 * 3, 1024),  # 256 channels from each ResNet branch, concatenated for 3 images
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_labels),  # Output final predictions
            nn.Sigmoid()  # Use Sigmoid for binary classification
        )

    def forward(self, img1, img2, img3):
        # Pass each image through the shared ResNet branch
        out1 = self.resnet_branch1(img1)
        out2 = self.resnet_branch2(img2)
        out3 = self.resnet_branch3(img3)

        # Global average pooling for each branch
        out1 = F.adaptive_avg_pool2d(out1, (1, 1))
        out2 = F.adaptive_avg_pool2d(out2, (1, 1))
        out3 = F.adaptive_avg_pool2d(out3, (1, 1))

        # Flatten the outputs
        out1 = out1.view(out1.size(0), -1)  # (batch_size, 256)
        out2 = out2.view(out2.size(0), -1)  # (batch_size, 256)
        out3 = out3.view(out3.size(0), -1)  # (batch_size, 256)

        # Concatenate the outputs from the 3 images
        out = torch.cat([out1, out2, out3], dim=1)  # (batch_size, 256 * 3)

        # Pass through the fully connected layers
        out = self.fc(out)
        return out

# Main Pipeline

## Pipeline Model

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, Optional

class PipelineSector(ABC):
  def __init__(self, input_is_list=False, ignore_input=False):
    self.input_is_list = input_is_list
    self.ignore_input = ignore_input

  @abstractmethod
  def __call__(self, input: Optional[Any]) -> Any:
    pass

class Pipeline(object):
  def __init__(self):
    self.pipe_list = []
  def append(self, sector:PipelineSector):
    self.pipe_list.append(sector)

  def run(self, x:Any) -> Any:
    for sector in self.pipe_list:
      print(f'[{sector.__class__.__name__}] Is Running')
      if sector.ignore_input:
        y = sector(None)
      elif type(x) is list and not sector.input_is_list:
        y = []
        for x_i in x:
          y.append(sector(x_i))
      else:
        y = sector(x)
      x = y
    return y


## Create DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from typing import Type, Any

class DataLoaderCreator(PipelineSector):
  def __init__(
    self,
    DS: Type[Dataset],
    ds_kwargs: dict[str, Any],
    dl_kwargs: dict[str, Any],
    with_eval = True,
    exclude_train = [],
    exclude_eval = []
  ):
    super().__init__(ignore_input=True)

    self.DS = DS
    self.ds_kwargs = ds_kwargs
    self.dl_kwargs = dl_kwargs
    self.with_eval = with_eval
    self.exclude_train = exclude_train
    self.exclude_eval = exclude_eval

  def __call__(self, root_dir:str):
    dataloaders = []

    if root_dir is None:
      root_dir = self.ds_kwargs.get('root_dir')
      del self.ds_kwargs['root_dir']

    dataset = self.DS(root_dir, exclude_list=self.exclude_train, **self.ds_kwargs)
    train_loader = DataLoader(dataset, **self.dl_kwargs)
    dataloaders.append(train_loader)


    if self.with_eval:
      dataset = self.DS(root_dir, exclude_list=self.exclude_eval, **self.ds_kwargs)
      eval_loader = DataLoader(dataset, **self.dl_kwargs)
      dataloaders.append(eval_loader)


    return dataloaders if self.with_eval else dataloaders[0]


## Model Trainer

In [ ]:
from torch.utils.data import DataLoader
from typing import Type, Any
import os
import pickle
import torch
import torch.nn as nn
from sklearn.metrics import recall_score, precision_score, accuracy_score
import numpy as np
from tqdm import tqdm

class ModelTrainer(PipelineSector):
  def __init__(
      self,
      Model: Type[nn.Module],
      device: torch.device,
      chunk_size:int,
      slide_size:int,
      model_path:str,
      save_path:str,
      model_name:str,
      patience,
      num_epochs,
      thresh,
      kwargs: dict[str, Any]
  ):
    super().__init__(input_is_list=True)

    self.model = Model(**kwargs)
    self.device = device
    self.chunk_size = chunk_size
    self.slide_size = slide_size
    self.save_path = save_path
    self.model_name = model_name
    self.patience = patience
    self.num_epochs = num_epochs
    self.thresh = thresh

    if model_path is not None:
      state_dict = torch.load(model_path, map_location=torch.device(device), weights_only=False)
      self.model.load_state_dict(state_dict)


  def initialize_metrics(self):
    return {
        'train_loss': [],
        'train_recall': [],
        'train_precision': [],
        'train_accuracy': [],
        'train_f1': [],
        'train_iou': [],
        'val_loss': [],
        'val_recall': [],
        'val_precision': [],
        'val_accuracy': [],
        'val_f1': [],
        'val_iou': []
    }

  def __call__(self, dataloaders: list[DataLoader]):
    save_path_ = os.path.join(self.save_path, self.model_name)
    last_model_path = os.path.join(self.save_path, 'last_model.pth')  # Path to save the last model

    train_loader = dataloaders[0]
    val_loader = dataloaders[1] if len(dataloaders)>1 else None


    self.model.to(self.device)

    pos_weight = torch.tensor([9.0]).to(self.device)  # Weight for the positive class (1)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # Use BCEWithLogitsLoss for stability
    optimizer = torch.optim.Adam(self.model.parameters(), lr=0.0001)

    # Metrics storage
    metrics = self.initialize_metrics()

    best_train_iou = float('-inf')  # Initialize best iou score for train
    best_val_iou = float('-inf')  # Initialize best iou score for validation

    best_model_state = None  # To store the best model state
    epochs_no_improve = 0  # Counter for epochs without improvement

    for epoch in range(self.num_epochs):
        # --- Training Phase ---
        self.model.train()
        running_loss = 0.0
        all_train_labels = []
        all_train_preds = []

        for i, ((img_before, img_current, img_after), labels) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}/{self.num_epochs}')):
            img_before = img_before.to(self.device)
            img_current = img_current.to(self.device)
            img_after = img_after.to(self.device)
            labels = labels.to(self.device)

            optimizer.zero_grad()
            outputs = self.model(img_before, img_current, img_after)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            all_train_labels.extend(labels.cpu().numpy())  # Store labels for metric calculation
            all_train_preds.extend((outputs.cpu().detach().numpy() > self.thresh).astype(float))  # Binarize predictions

            if (i + 1) % 10 == 0:  # Print every 10 batches
                print(f'Epoch [{epoch + 1}/{self.num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / (i + 1):.4f}')

        # Calculate training metrics for the epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_recall = recall_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
        epoch_precision = precision_score(all_train_labels, all_train_preds, average='macro', zero_division=0)
        epoch_accuracy = accuracy_score(all_train_labels, all_train_preds)
        epoch_f1 = 2 * (epoch_precision * epoch_recall) / (epoch_precision + epoch_recall + 1e-7)  # f1 calculation

        intersection = np.logical_and(all_train_labels, all_train_preds).sum()
        union = np.logical_or(all_train_labels, all_train_preds).sum()
        epoch_iou = intersection / float(union) if union > 0 else 0

        # Store training metrics
        metrics['train_loss'].append(epoch_loss)
        metrics['train_recall'].append(epoch_recall)
        metrics['train_precision'].append(epoch_precision)
        metrics['train_accuracy'].append(epoch_accuracy)
        metrics['train_f1'].append(epoch_f1)
        metrics['train_iou'].append(epoch_iou)

        print(f'Epoch [{epoch+1}/{self.num_epochs}] complete. Train Loss: {epoch_loss:.4f}, Train Recall: {epoch_recall:.4f}, Train Precision: {epoch_precision:.4f}, Train Accuracy: {epoch_accuracy:.4f}, Train f1: {epoch_f1:.4f}, Train iou: {epoch_iou:.4f}')

        # --- Validation Phase ---
        if val_loader:
            self.model.eval()
            val_loss = 0.0
            all_val_labels = []
            all_val_preds = []

            with torch.no_grad():
                for (img_before, img_current, img_after), labels in tqdm(val_loader, desc="Validation"):
                    img_before = img_before.to(self.device)
                    img_current = img_current.to(self.device)
                    img_after = img_after.to(self.device)
                    labels = labels.to(self.device)

                    outputs = self.model(img_before, img_current, img_after)
                    loss = criterion(outputs.squeeze(), labels.float())

                    val_loss += loss.item()
                    all_val_labels.extend(labels.cpu().numpy())  # Store labels for metric calculation
                    all_val_preds.extend((outputs.cpu().detach().numpy() > 0.5).astype(float))  # Binarize predictions

            # Calculate validation metrics for the epoch
            val_loss /= len(val_loader)
            val_recall = recall_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
            val_precision = precision_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
            val_accuracy = accuracy_score(all_val_labels, all_val_preds)
            val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall + 1e-7)

            intersection = np.logical_and(all_val_labels, all_val_preds).sum()
            union = np.logical_or(all_val_labels, all_val_preds).sum()
            val_iou = intersection / float(union) if union > 0 else 0

            # Store validation metrics
            metrics['val_loss'].append(val_loss)
            metrics['val_recall'].append(val_recall)
            metrics['val_precision'].append(val_precision)
            metrics['val_accuracy'].append(val_accuracy)
            metrics['val_f1'].append(val_f1)
            metrics['val_iou'].append(val_iou)

            print(f'Validation complete. Val Loss: {val_loss:.4f}, Val Recall: {val_recall:.4f}, Val Precision: {val_precision:.4f}, Val Accuracy: {val_accuracy:.4f}, Val f1: {val_f1:.4f}, Val iou: {val_iou:.4f}')

        # Save metrics and models
        history_path = os.path.join(self.save_path, f'{self.model_name[:-4]}.pkl')
        with open(history_path, 'wb') as f:
            pickle.dump(metrics, f)


        if epoch_iou > best_train_iou:  # Train IoU
            best_train_iou = epoch_iou
            best_model_state = self.model.state_dict()  # Save the best model's state dict based on train IoU
            torch.save(best_model_state, os.path.join(self.save_path, f"best_model_train_iou.pth"))  # Save with train IoU tag
            print(f"New best model found at epoch {epoch+1} with train IoU score: {epoch_iou:.4f}. Saving model.")

        # Early stopping and best model saving based on validation iou
        if val_loader and val_iou > best_val_iou:
            best_val_iou = val_iou
            best_model_state = self.model.state_dict()  # Save the best model's state dict
            torch.save(best_model_state, save_path_)  # Save the best model
            print(f"New best model found at epoch {epoch+1} with iou score: {val_iou:.4f}. Saving model.")
            epochs_no_improve = 0  # Reset the counter if iou improved
        else:
            epochs_no_improve += 1
            print(f"No improvement in iou score for {epochs_no_improve} epoch(s).")

        # Save the last model at the end of each epoch
        torch.save(self.model.state_dict(), last_model_path)
        print(f"Last model saved at epoch {epoch+1}.")

        if epochs_no_improve >= self.patience:
            print(f"Early stopping at epoch {epoch+1}. Best iou score: {best_val_iou:.4f}")
            break

    return metrics


## Model Loader

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from typing import Type, Any
import copy
from tqdm import tqdm

class ModelLoader(PipelineSector):
  def __init__(
      self,
      Model: Type[nn.Module],
      model_path:str,
      device: torch.device,
      chunk_size:int,
      slide_size:int,
      thresh:float,
      kwargs: dict[str, Any]
  ):
    super().__init__()

    self.model = Model(**kwargs)
    self.device = device
    self.chunk_size = chunk_size
    self.slide_size = slide_size
    self.thresh=thresh

    state_dict = torch.load(model_path, map_location=torch.device(device), weights_only=False)

    self.model.load_state_dict(state_dict)


  def pad_beg_end(self, L:list, length:int=15):
    # Create the padding list
    z = [0.0] * length

    # Create a copy of the list
    L_copy = L.copy()

    # Insert padding at the beginning and append it at the end
    L_copy.insert(0, z)
    L_copy.append(z)

    # Return the padded copy
    return L_copy


  def average_lists(self, list1:list, list2:list):
      """A helper function to average two lists element-wise."""
      return [(a + b) / 2 for a, b in zip(list1, list2)]


  def sliding_window_operation(self, data:list):
      # Create a deep copy of the data to avoid changing the original
      data_copy = copy.deepcopy(data)

      # Ensure we have at least one window to process
      if len(data_copy) < 2:
          raise ValueError("Data must contain at least two elements.")

      i = 0
      while len(data_copy) >= 2:
          # Get the current window
          current_window = data_copy[0:2]
          # print('Current:', current_window)

          len_0 = len(current_window[0])
          len_1 = len(current_window[1])

          window_0_last = current_window[0][-self.chunk_size:]
          window_1_first = current_window[1][-self.chunk_size:]
          average_first = self.average_lists(window_0_last, window_1_first)


          # First
          window_0_first = current_window[0][:(len_0 - self.chunk_size)]
          window_1_last = current_window[0][-self.slide_size:]

          new = window_0_first + average_first + window_1_last

          # Replace the two elements with the new list
          data_copy[0: 2] = [new]

      return data_copy


  def __call__(self, dataloader: DataLoader):
    list_of_outputs = []

    self.model.eval()
    self.model.to(self.device)

    for i,(img_before, img_current, img_after) in enumerate(tqdm(dataloader, desc='Making Prediction')):
      img_before = img_before.to(self.device)
      img_current = img_current.to(self.device)
      img_after = img_after.to(self.device)

      ot = self.model(img_before, img_current, img_after)
      ot = ot.flatten()
      ot = ot.detach().cpu().numpy()
      ot = list(ot)
      list_of_outputs.append(ot)


    padded_list = self.pad_beg_end(list_of_outputs, self.chunk_size)
    labels = np.array(self.sliding_window_operation(padded_list), dtype=float).flatten()

    labels[labels > self.thresh] = 1
    labels[labels <= self.thresh] = 0

    return labels


# Train Pipeline

In [ ]:
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the Mel spectrogram
    transforms.ToTensor(),  # Convert to PyTorch tensor
])

pipe = Pipeline()

pipe.append(WavLoader())
pipe.append(ChunksCreator(
    'chunks',
    NoiseCancalation(),
    MelSpectrogram(),
    exist_ok=False
))
pipe.append(DataLoaderCreator(
    MelSpectrogramDataset,
    {
        'chunk_size': 15,
        'sample_rate': 44100,
        'transform': data_transform,
        'root_dir': 'chunks'
    },
    {
        'batch_size': 32,
        'shuffle': False
    }
))
pipe.append(ModelTrainer(
    Model=MultiInputResNet,
    device='cpu',
    chunk_size=15,
    slide_size=5,
    model_path=None,
    save_path='.',
    model_name='best.pth',
    patience=3,
    num_epochs=10,
    thresh=0.193,
    kwargs={
        'num_labels': 15
    }
))

# Test Pipeline

In [ ]:
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize the Mel spectrogram
    transforms.ToTensor(),  # Convert to PyTorch tensor
])

pipe = Pipeline()

# pipe.append(WavDownloader(
#     root_dir='test',
#     file_names=[
#         'S2.wav',
#         'S6.wav',
#         'S12.wav'
#     ]
# ))
pipe.append(WavLoader())
pipe.append(ChunksCreator(
    'chunks',
    NoiseCancalation(),
    MelSpectrogram(),
    exist_ok=False,
    with_labels=False
))
pipe.append(DataLoaderCreator(
    DS=MelSpectrogramDataset,
    with_eval=False,
    ds_kwargs={
        'chunk_size': 15,
        'sample_rate': 44100,
        'transform': data_transform,
        'root_dir': 'chunks'
    },
    dl_kwargs={
        'batch_size': 32,
        'shuffle': False
    }
))
pipe.append(ModelLoader(
    Model=MultiInputResNet,
    device='cpu',
    chunk_size=15,
    slide_size=5,
    model_path='/content/drive/MyDrive/AIA/public AIA/fold_3_kaggle_val.pth',
    thresh=0.193,
    kwargs={
        'num_labels': 15
    }
))
pipe.append(Labeler(
    min_length_ones=10
))

In [ ]:
# pipe.run([
#     'http://dl.iaievent.com/Challenge1/Test_S2.wav',
#     'http://dl.iaievent.com/Challenge1/Test_S6.wav',
#     'http://dl.iaievent.com/Challenge1/Test_S12.wav'
# ])
res = pipe.run('test')

[WavLoader] Is Running
[ChunksCreator] Is Running
[DataLoaderCreator] Is Running
[ModelLoader] Is Running


Making Prediction: 0it [00:00, ?it/s]

[Labeler] Is Running


In [ ]:
res

[['0:00', '0:00']]